In [1]:
import os
from pprint import pprint

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Neo4j@123"

In [2]:
# testing the graph connectivity 
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()
graph.schema

'Node properties are the following:\nDetections {unique_vendor_detection_id: STRING, vendor_id: STRING, type: STRING, severity: STRING, status: STRING, first_found_datetime: STRING, last_found_datetime: STRING, detection_id: STRING},Qualys_Knowledge_Base {vulnerability_type: STRING, severity_level: STRING, title: STRING, category: STRING, publised_datetime: STRING, patchable: STRING, diagnosis: STRING, pci_flag: STRING, consequences: STRING, solution: STRING, qualys_id: STRING, cve_ids: STRING},Hosts {ip: STRING, os: STRING, hostname: STRING, domain: STRING, netbios: STRING, last_scan_datetime: STRING, asset_id: STRING},Assets {ip: STRING, os: STRING, hostname: STRING, domain: STRING, netbios: STRING, last_scan_datetime: STRING, asset_id: STRING},CVE_DATA {date_published: STRING, description: STRING, cve_id: STRING, version: STRING, baseScore: STRING, vectorString: STRING, attackComplexity: STRING, attackVector: STRING, availabilityImpact: STRING, baseSeverity: STRING, confidentialityI

In [6]:
# query on graph 

from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
import os 


graph = Neo4jGraph()
llm = ChatOpenAI(model="gpt-4", temperature=0)
chain = GraphCypherQAChain.from_llm(graph = graph, llm = llm, verbose=True, validate_cypher=True)


In [5]:
print(chain.graph_schema)

Node properties are the following:
Detections {unique_vendor_detection_id: STRING, vendor_id: STRING, type: STRING, severity: STRING, status: STRING, first_found_datetime: STRING, last_found_datetime: STRING, detection_id: STRING},Qualys_Knowledge_Base {vulnerability_type: STRING, severity_level: STRING, title: STRING, category: STRING, publised_datetime: STRING, patchable: STRING, diagnosis: STRING, pci_flag: STRING, consequences: STRING, solution: STRING, cve_ids: STRING, qualys_id: STRING},Hosts {ip: STRING, os: STRING, hostname: STRING, domain: STRING, netbios: STRING, last_scan_datetime: STRING, asset_id: STRING},Assets {ip: STRING, os: STRING, hostname: STRING, domain: STRING, netbios: STRING, last_scan_datetime: STRING, asset_id: STRING},CVE_DATA {date_published: STRING, description: STRING, cve_id: STRING, version: STRING, baseScore: STRING, vectorString: STRING, attackComplexity: STRING, attackVector: STRING, availabilityImpact: STRING, baseSeverity: STRING, confidentialityImp

In [7]:
# question 1 
response = chain.invoke({"query": "what do you know about CVE-2023-2021"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:CVE_DATA {cve_id: 'CVE-2023-2021'}) RETURN c
Full Context:
[{'c': {'availabilityImpact': 'LOW', 'description': "Cross-site Scripting (XSS) - Stored in GitHub repository nilsteampassnet/teampass prior to 3.0.3.. CWE-79 Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'). version = 3.0 . vectorString = CVSS:3.0/AV:L/AC:H/PR:N/UI:R/S:U/C:H/I:L/A:L . attackVector = LOCAL . attackComplexity = HIGH . privilegesRequired = NONE . userInteraction = REQUIRED . scope = UNCHANGED . confidentialityImpact = HIGH . integrityImpact = LOW . availabilityImpact = LOW . baseScore = 5.8 . baseSeverity = MEDIUM .", 'baseScore': '5.8', 'privilegesRequired': 'NONE', 'version': '3.0', 'userInteraction': 'REQUIRED', 'baseSeverity': 'MEDIUM', 'confidentialityImpact': 'HIGH', 'date_published': '2023-04-13T00:00:00', 'cve_id': 'CVE-2023-2021', 'attackComplexity': 'HIGH', 'scope': 'UNCHANGED', 'attackVector': 'L

In [10]:
response = chain.invoke({'query': "How many vulnerability findings  were discovered in 2024?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:Findings)
WHERE f.first_found_datetime CONTAINS '2024'
RETURN COUNT(f)
Full Context:
[{'COUNT(f)': 160}]

> Finished chain.
{'query': 'How many vulnerability findings  were discovered in 2024?',
 'result': '160 vulnerability findings were discovered in 2024.'}


In [11]:
response = chain.invoke({'query': "How many vulnerability findings  were discovered in 2024? Group them by severity."})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:Findings)
WHERE f.first_found_datetime CONTAINS '2024'
RETURN f.severity, COUNT(*) AS count
ORDER BY count DESC
Full Context:
[{'f.severity': '4', 'count': 89}, {'f.severity': '3', 'count': 35}, {'f.severity': '2', 'count': 22}, {'f.severity': '5', 'count': 14}]

> Finished chain.
{'query': 'How many vulnerability findings  were discovered in 2024? Group '
          'them by severity.',
 'result': 'In 2024, 89 vulnerability findings of severity 4, 35 of severity '
           '3, 22 of severity 2, and 14 of severity 5 were discovered.'}


In [12]:
response = chain.invoke({'query': "How many vulnerabilities were discovered by Qualys in 2024?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Detections)-[:MORE_INFORMATION]->(q:Qualys_Knowledge_Base)-[:MORE_INFORMATION]->(v:Vulnerabilities) 
WHERE d.vendor_id = "Qualys" AND d.first_found_datetime CONTAINS "2024" 
RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 0}]

> Finished chain.
{'query': 'How many vulnerabilities were discovered by Qualys in 2024?',
 'result': 'No vulnerabilities were discovered by Qualys in 2024.'}


In [5]:
# question 2 
# which vulnerabilities are we talking about?
response = chain.invoke({'query': "How many vulnerabilities were discovered in 2024?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vulnerabilities) WHERE v.date_published STARTS WITH '2024' RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 11355}]

> Finished chain.
{'query': 'How many vulnerabilities were discovered in 2024?',
 'result': '11355 vulnerabilities were discovered in 2024.'}


In [6]:
# question 2 
# response = chain.invoke({'query': "How many  vulnerabilities were discovered in 2024?"})
response = chain.invoke({'query': "How many cve vulnerabilities were discovered in 2024?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:CVE_DATA) WHERE v.date_published CONTAINS '2024' RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 11355}]

> Finished chain.
{'query': 'How many cve vulnerabilities were discovered in 2024?',
 'result': '11355 CVE vulnerabilities were discovered in 2024.'}


In [8]:
# question 3 
response = chain.invoke({'query': "Which vulnerabilities are exploitable by ransomware?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Ransomeware)-[:EXPLOITS]->(v:Vulnerabilities) RETURN v
Full Context:
[{'v': {'availabilityImpact': 'HIGH', 'description': 'Vulnerability in the Oracle WebLogic Server component of Oracle Fusion Middleware (subcomponent: Web Services). Supported versions that are affected are 10.3.6.0.0 and 12.1.3.0.0. Easily exploitable vulnerability allows unauthenticated attacker with network access via HTTP to compromise Oracle WebLogic Server. Successful attacks of this vulnerability can result in takeover of Oracle WebLogic Server. CVSS 3.0 Base Score 9.8 (Confidentiality, Integrity and Availability impacts). CVSS Vector: (CVSS:3.0/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H).. Easily exploitable vulnerability allows unauthenticated attacker with network access via HTTP to compromise Oracle WebLogic Server.  Successful attacks of this vulnerability can result in takeover of Oracle WebLogic Server.. attackComplexity = LOW . attackVecto

In [9]:
# question 4 
response = chain.invoke({'query': "Which vulnerabilities are exploitable by malware?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Malware)-[:EXPLOITS]->(v:Vulnerabilities) RETURN v.title, v.vulnerability_type, v.severity_level, v.category, v.publised_datetime, v.patchable, v.diagnosis, v.pci_flag, v.consequences, v.solution, v.cve_ids, v.qualys_id, v.date_published, v.description, v.cve_id, v.version, v.baseScore, v.vectorString, v.attackComplexity, v.attackVector, v.availabilityImpact, v.baseSeverity, v.confidentialityImpact, v.integrityImpact, v.privilegesRequired, v.scope, v.userInteraction, v.exploitCodeMaturity, v.remediationLevel, v.reportConfidence, v.temporalScore, v.temporalSeverity, v.type, v.content
Full Context:
[{'v.title': None, 'v.vulnerability_type': None, 'v.severity_level': None, 'v.category': None, 'v.publised_datetime': None, 'v.patchable': None, 'v.diagnosis': None, 'v.pci_flag': None, 'v.consequences': None, 'v.solution': None, 'v.cve_ids': None, 'v.qualys_id': None, 'v.date_published': '2022-10-03T16:15:28', 'v.descript

In [8]:
# question 5
response = chain.invoke({'query': "How many vulnerabilities don't have a patch available?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vulnerabilities) WHERE v.patchable = "No" RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 12065}]

> Finished chain.
{'query': "How many vulnerabilities don't have a patch available?",
 'result': "There are 12,065 vulnerabilities that don't have a patch "
           'available.'}


In [9]:
# question 6
# is this correct?
response = chain.invoke({'query': "How many vulnerabilities have a exploit available?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vulnerabilities)-[:EXPLOITED_BY]->() RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 5630}]

> Finished chain.
{'query': 'How many vulnerabilities have a exploit available?',
 'result': 'There are 5630 vulnerabilities that have an exploit available.'}


In [10]:
# question 7
response = chain.invoke({'query': "What is severity distribution (high, medium, low, critical) of vulnerabilities discovered in 2022?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vulnerabilities) 
WHERE v.publised_datetime >= "2022-01-01" AND v.publised_datetime < "2023-01-01" 
RETURN v.severity_level AS Severity, COUNT(*) AS Distribution 
ORDER BY Distribution DESC
Full Context:
[{'Severity': '4', 'Distribution': 15013}, {'Severity': '3', 'Distribution': 9963}, {'Severity': '5', 'Distribution': 4347}, {'Severity': '2', 'Distribution': 2287}, {'Severity': '1', 'Distribution': 70}]

> Finished chain.
{'query': 'What is severity distribution (high, medium, low, critical) of '
          'vulnerabilities discovered in 2022?',
 'result': 'The distribution of vulnerabilities discovered in 2022 is as '
           'follows: 15013 with high severity (4), 9963 with medium severity '
           '(3), 4347 with critical severity (5), 2287 with low severity (2), '
           'and 70 with the least severity (1).'}


In [11]:
# question 8
# is this specifically related to Qualys?
response = chain.invoke({'query': "How many PCI related vulnerabilities were reported in 2023?"})
pprint(response)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Vulnerabilities) WHERE v.pci_flag = "true" AND v.publised_datetime CONTAINS "2023" RETURN COUNT(v)
Full Context:
[{'COUNT(v)': 0}]

> Finished chain.
{'query': 'How many PCI related vulnerabilities were reported in 2023?',
 'result': 'No PCI related vulnerabilities were reported in 2023.'}


In [ ]:
# result 4 of 8 
# area of improvement 
# creation of graph - convert values from flags to text 

In [2]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

existing_graph = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url="bolt://localhost:7687",
    username="neo4j",
    password="Neo4j@123",
    index_name="software_name_index",
    node_label="Software",
    text_node_properties=["product", "vendor"],
    embedding_node_property="embedding",
)

In [5]:
existing_graph.similarity_search("Apache web server", k = 100)

[Document(page_content='\nproduct: webserver\nvendor: apache', metadata={'software_id': 'apache.webserver', 'vulnerability_id': '6d8821cb-8dcd-4d15-91a1-1ba2edc6819c'}),
 Document(page_content='\nproduct: http server\nvendor: apache', metadata={'software_id': 'apache.http server', 'vulnerability_id': 'bfef03d2-b119-46dd-9d14-0bb05d8153c0'}),
 Document(page_content='\nproduct: apache httpd\nvendor: apache', metadata={'software_id': 'apache.apache httpd', 'vulnerability_id': '76c56545-b398-4f7e-a9af-e9c7aca8fefa'}),
 Document(page_content='\nproduct: httpd\nvendor: apache', metadata={'software_id': 'apache.httpd', 'vulnerability_id': '042f6008-d41f-4bcc-bf6b-0f60a3bca495'}),
 Document(page_content='\nproduct: apache\nvendor: ubuntu', metadata={'software_id': 'ubuntu.apache', 'vulnerability_id': '17098100-77e8-4b4c-92f1-076d6f1410ab'}),
 Document(page_content='\nproduct: apache_http_server\nvendor: apache', metadata={'software_id': 'apache.apache_http_server', 'vulnerability_id': '240c02a

**Fields to be Embedded**

1. Qualys_Knowldge_Base 
    - consequences
    - diagnosis 
    - solution

2. CVE_DATA
    - description

3.  